In [ ]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../../..'))

import importlib

import contextlib

import json
import os
import random

import numpy as np
import torch
from collections import defaultdict
from utils import get_default_path
import time
from utils import Stopwatch
import datetime

import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import clip
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
prompt_dataframe = pd.DataFrame({
    "prompt": [],
    "cfg": [],
    "width": [],
    "height": [],
    "num_imgs": [],
    "sampler": [],
    "is_grid": [],
    "steps": [],
    "seed": [],
    "urls": [],
    "date": [],
    "user": [],
}) 

In [2]:
sd_discord = get_default_path("sd_discord")
json_paths = [path for path in os.listdir(sd_discord) if "gobot" in path and path.endswith(".json")]
channels = {}

def is_prompt_message(msg):
    return msg["author"]["isBot"] and "content" in msg and msg["content"].startswith("Dreamt")


for file_name in json_paths:
    file_path = os.path.join(sd_discord, file_name)
    
    with open(file_path, 'r') as discord_file:
        json_dict = json.load(discord_file)
        channel_id = json_dict["channel"]["id"]
        guid = json_dict["guild"]["id"]
        channels[(guid, channel_id)] = [msg for msg in json_dict["messages"] if msg["author"]["isBot"]]

In [ ]:
sd_discord = get_default_path("discord_joe")
json_paths = [path for path in os.listdir(sd_discord) if path.startswith("Stable") and path.endswith(".json")]
channels = {}


for file_name in json_paths:
    file_path = os.path.join(sd_discord, file_name)
    print(f"Parsing {file_path}")
    
    try:
        with open(file_path, 'r') as discord_file:
            json_dict = json.load(discord_file)
            channel_id = json_dict["channel"]["id"]
            guid = json_dict["guild"]["id"]
            channels[(guid, channel_id)] = [msg for msg in json_dict["messages"] if msg["author"]["isBot"]]
    except Exception as ex:
        print(f"failed to load {file_path}: {ex}")

In [ ]:
json_dict.keys()

In [23]:
import re
import time
from ai.stabledisco.prompts.stablediscoprompt import StableDiscoPrompt
import traceback
seen = set()

num_no_mention = 0
def add_to_rows(seed_to_msg, discord_json, guid, channel_id, dry_run=False):
    global num_no_mention
    content = discord_json["content"].lower()
    multi_command_idx = content.find("The commands")
    if multi_command_idx != -1:
        content = content[:multi_command_idx]
        

    
    if not discord_json["attachments"] or  not discord_json["author"]["isBot"] or "!help" in content:
        return
    
    if not discord_json["mentions"]:
        num_no_mention += 1
        if num_no_mention < 15:
            print("No mention")
            print(content)
        return
        
    content = re.sub(r"\s", " ", content)
    if not content:
        return
    prompt_str = re.findall(r'(?<=!dream )".*".*-.*(?=`)', content)
    if not prompt_str:
        prompt_str = re.findall(r'(?<=!dream ").*(?=" -)',content )
    if not prompt_str:
        prompt_str = re.findall(r'(?<=dreamt of ").*(?=")',content )
    if not prompt_str:
        prompt_str = re.findall(r'(?<=/dream ).*(?=`)', content)
    if not prompt_str:
        prompt_str = re.findall(r'(?<=!dream).*', content)
    if not prompt_str:
        prompt_str = re.findall(r'(?<=dreamt of).*',content )
    if not prompt_str:
        print(f"Failed to extract {content}")
        print()
        return
    prompt_str = prompt_str[0]
    cut_idx = re.search(r"[` ]*(\*\*|\*\*WARNING\*\*|The seeds for each individual image are)", prompt_str) 

    if cut_idx:
        cut_idx = cut_idx.span()[0]
        prompt_str = prompt_str[:cut_idx]
    try:
        command = StableDiscoPrompt.from_str(prompt_str) 
    except Exception as ex:
        print(f"Can't extract prompt from {prompt_str}\n full str {content}")
        print(ex)
        print()
        time.sleep(0.05)
        return
        
    prompt = command.prompt
    prompt = prompt.replace("prompt:", "")
    
    first_quote = prompt.find("\"")
    if first_quote != -1:
        prompt = prompt[:first_quote] + prompt[first_quote+1:]

        last_quote = prompt.rfind("\"")
        prompt = prompt[:last_quote] + prompt[last_quote+1:]
        prompt = prompt.strip()
    
    
    seeds = []
    seed_matches = re.finditer("(?<=each individual image are: )\[[\d+, ]*\]", content)
    for match in seed_matches:
        seeds = eval(match.group(0))

    seen.add(prompt)
    
    if not seeds:
        seed_matches = re.finditer("(?<=seed:)[\d+ ]*", content)
        for match in seed_matches:
            seeds = [int(match.group(0))]
            
    
    if not seeds:
        try:
            if "-s" in command.args:
                seeds = [command.args["-s"]]
            else:
                seed_matches = re.finditer("(?<=-s )\d+", prompt)

                for match in seed_matches:
                    seeds = [int(match.group(0))]
                
        except:
            print("Failed to get seed")
            print(prompt)
    

    user = discord_json["mentions"][0]["name"]
    
    seeds = [int(seed) for seed in seeds]
    
    link = f"https://discord.com/channels/{guid}/{channel_id}/{discord_json['id']}"
    
    url_seeds = []
    for attachement in discord_json["attachments"]:
        msg = attachement["url"]
        seed_matches = re.finditer("(?<=_seed-)[\d+]*", content)
        for match in seed_matches:
            url_seeds.append(int(match.group(0)))
    
    message = {
        "author": user,
        "link": link,
        "prompt": command.get_prompt_with_args(),
        "filenames":  [attachement["fileName"] for attachement in discord_json["attachments"]],
        "urls":  [attachement["url"] for attachement in discord_json["attachments"]],
    }

    
    for seed in seeds:
        seed_to_msg[seed].append(message)


from collections import defaultdict
seed_to_msg = defaultdict(list)

idx = 0
for ids, messages in channels.items():
    guid, channel_id = ids
    for msg in messages:
        if idx != 0 and idx % 100000 == 0:
            print(f"Finished {idx} of {len(messages)}")
        try:
            add_to_rows(seed_to_msg, msg, guid, channel_id)
        except Exception as ex:
            print(traceback.format_exc())
            time.sleep(0.2)
        idx += 1

Finished 100000 of 103646
Finished 200000 of 18963
Finished 300000 of 52308
Finished 400000 of 39478
Finished 500000 of 42999
Finished 600000 of 38364
Finished 700000 of 104953
Finished 800000 of 61728
Finished 900000 of 52225
Finished 1000000 of 44343
Finished 1100000 of 43919
Finished 1200000 of 56178
No mention
dreamt in `88.91s` for @scrdnki's `!dream "realistic detailed image of a big cockroach sleeping in a bed in an old soviet apartment bedroom,, fully clothed boy plays a computer game, by ayami kojima, amano, karol bak, neo - gothic, gothic, rich deep colors. beksinski painting, from a movie by david cronenberg. francis bacon masterpiece. " -h 768 -w 1024 -c 13.0 -n 9 -i -s 150 -s 2897720055` the seeds for each individual image are: [2897720055, 3621375592, 3410446568, 1581896163, 3574994417, 3388684419, 40571012, 589575177, 2713667972]
Finished 1300000 of 47889
Finished 1400000 of 47125
No mention
dreamt in `14.45s` for @desm0nt's `!dream "fullbody portrait of anthropomorphic 

In [82]:
seed_to_msg_dict = dict(seed_to_msg)

with open(get_default_path("discord_dict", "sd_discord_messages_trimed.json"),'w+') as outfile:
    json.dump(seed_to_msg_dict, outfile)

In [80]:
for key in seed_to_msg_dict.keys():
    for msg in seed_to_msg_dict[key]:
        try:
            del msg["filenames"]
        except:
            pass

In [52]:
total_imgs = 0
cnt_to_seed = []

for key, val in seed_to_msg_dict.items():
    cnt = len(val)
    total_imgs += cnt
    cnt_to_seed.append((cnt, key))

In [53]:
cnt_to_seed.sort()

In [63]:
print('Num seeds :', len(cnt_to_seed))
print('Num images:', total_imgs)

num_to_print = 20
print(f"{num_to_print} most popular seeds")
for idx in range(-1, -num_to_print, -1):
    pair = cnt_to_seed[idx]
    print(f"{pair[1]:10d}: {pair[0]}")



Num seeds : 10619239
Num images: 11913435
20 most popular seeds
       150: 2621
3508342747: 2381
2726823234: 2287
3243157889: 2257
2615227870: 2146
3411521810: 2093
         1: 2085
4101097972: 1880
2556321752: 1828
2013010495: 1775
2854263694: 1269
4039455774: 943
2596996162: 911
1879968118: 854
        42: 816
1823804162: 814
       100: 781
2949882636: 735
1729261567: 734


In [74]:
import PIL
import requests
def load_img(url: str):
    try:
        url_match = re.search("(?P<url>https?://[^\s]+)", url)
        if url_match:
            url = url_match.group("url")
        else:
            print(f"Url {url} appears invalid")
            return

        return PIL.Image.open(
            requests.get(url, stream=True, timeout=10).raw
        ).convert("RGB")

    except Exception as ex:
        print(f"Unable to display url {url}: {ex}")
    
def process_urls(urls, seed, display_imgs):
    if len(urls) == 1:
        url = urls[0]
        print(f"Url: {url}")
        if display_imgs:
            display(load_img(url))
        return
    for url in urls:
        url = url.replace(f"-S_{seed}", "")
        url = url.replace(f"-s_{seed}", "")
        if str(seed) in url:
            print(f"Url: {url}")
            if display_imgs:
                display(load_img(url))
            return
        

def author_for_seed(seed_map, seed, filter_words, filter_threshold=10, display_imgs=False):
    msgs = seed_map[seed]

    if not msgs:
        print(f"No messages found for seed {seed}")
        return
    
    if len(msgs) == 1:
        author = msgs[0]["author"]
        prompt = msgs[0]["prompt"]
        link = msgs[0]["link"]
        print(f"One message found for seed {seed}")
        print(f"User: {author}")
        print(f"Discord Link: {link}")
        print(f"Prompt: {prompt}")
        process_urls(msgs[0]["urls"], seed, display_imgs=display_imgs)
                    
        return
                
    print(f"{len(msgs)} images found for seed {seed}")
    author_to_cnt = defaultdict(int)
    for msg in msgs:
        author_to_cnt[msg["author"]] += 1
    num_authors = len(author_to_cnt)
    print(f"Num authors: {num_authors}")
    if not filter_words or num_authors < filter_threshold:
        msg_to_scan = msgs
    else:
        msg_to_scan = []
    
        for msg in msgs:
            found = False
            for word in filter_words:
                if word.lower() in msg["prompt"].lower():
                    found = True
                    msg_to_scan.append(msg)
                    break
            if not found:
                continue
                
        author_to_cnt = defaultdict(int)
        for msg in msg_to_scan:
            author_to_cnt[msg["author"]] += 1
        num_authors = len(author_to_cnt)
        print(f"Num authors with matching words: {num_authors}")
    
    for msg in msg_to_scan:
        author = msg["author"]
        prompt = msg["prompt"]
        link = msg["link"]
            
        print(f"User: {author}")
        print(f"Discord Link: {link}")
        print(f"Prompt: {prompt}")
        process_urls(msg["urls"], seed, display_imgs=display_imgs)
        print()

filter_words = []
author_for_seed(seed_to_msg, 420, filter_words)

153 images found for seed 420
Num authors: 15
User: OCBeerSociety
Discord Link: https://discord.com/channels/1002292111942635562/1005512133070557194/1006958769344872468
Prompt: detailed photograph of napoleon dynamite as dr who   -s 150
Url: https://cdn.discordapp.com/attachments/1005512133070557194/1006958768891891762/detailed_photograph_of_Napoleon_Dynamite_as_Dr_Who_-n_9_-g_-s_150_-S_420_ts-1660148074_idx-0.png

User: BRi7X
Discord Link: https://discord.com/channels/1002292111942635562/1005596500845404301/1009304355347898388
Prompt: close - up shot four jacks face cards, painted in the style of pointilism, extremely highly detailed  -h 832 -w 832
Url: https://cdn.discordapp.com/attachments/1005596500845404301/1009304355075264582/close_-_up_shot_four_jacks_face_cards_painted_in_the_style_of_pointilism_extremely_highly_detailed_-H_832_-W_832_-S_420_ts-1660707309_idx-0.png

User: LeLas
Discord Link: https://discord.com/channels/1002292111942635562/1005596500845404301/100948503902265761

In [ ]:
seed_to_msg[3640807225]

In [ ]:
for row in discord_prompt_dataframe.iterrows():
    

In [ ]:
prompt_dataframe = pd.DataFrame.from_records(rows)

In [ ]:
len(prompt_dataframe.loc[prompt_dataframe['cfg'] != 7.0])

In [ ]:
#file_path = os.path.join(sd_discord, file_name)
prompt_dataframe.to_csv(file_path)

In [ ]:
prompt_dataframe.describe()

In [ ]:
prompt_dataframe['img_path'] = ""

In [ ]:
print(prompt_dataframe.loc[0])

In [ ]:
import shutil
import requests

def save_img(url, path):
    prompt_dataframe.loc[idx,"urls"]
    start = url.find("['") + 2
    end = url.find("'",start+1)
    url = url[start:end]
    response = requests.get(url, stream=True)
    with open(path, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

import requests
img_dir = "sd_discord/imgs"

for idx in prompt_dataframe.index:
    if idx % 10000 == 0:
        print(f"{idx} of {len(prompt_dataframe)} finished")
    img_path = get_default_path(img_dir, f"{idx}.png")
    save_img(prompt_dataframe.loc[idx,"urls"], img_path)
    prompt_dataframe.loc[idx, 'img_path'] = img_path

In [ ]:
prompt_dataframe.to_csv(file_path)

In [ ]:
print(len(prompt_dataframe[prompt_dataframe['img_path'] == ""]))